# Nycflights2013 Analysis

In [1]:
install.packages("nycflights13")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [8]:
library(tidyverse)
library(glue)
library(nycflights13)
library(lubridate)
library(dplyr)



# **Question** 1 : Find Top 5 probably high fuel expense. 
 If **distance** is one factor in calculating the fuel expenses

In [3]:
##Prepare Data
# Find cancelled flights 
not_cancelled <- filter(flights,!is.na(arr_time),!is.na(air_time))

# Find amount of miles from distance column 
df1 <- not_cancelled %>%
  group_by(carrier) %>%
  summarise(miles = sum(distance),
            count = n()) %>%
  mutate(flight_rank=round(count/sum(count)*100,digits = 2)) %>%
  mutate(mile_rank=round(miles/sum(miles)*100,digit = 2)) %>%
  mutate(miles = round(miles/1000000),digit = 2) %>%
  arrange(desc(miles))

# Find top 5 probably high fuel expense. 
# If distance is one factor in calculating the fuel expenses
left_join(df1,airlines, by = "carrier") %>%
  select(carrier,name,miles) %>%
  rename("miles(million)" = "miles") %>%  
  head(5)

carrier,name,miles(million)
<chr>,<chr>,<dbl>
UA,United Air Lines Inc.,88
DL,Delta Air Lines Inc.,59
B6,JetBlue Airways,58
AA,American Airlines Inc.,43
EV,ExpressJet Airlines Inc.,29


**Description :** If distance is one factor in calculating the fuel expenses. That means total distance of each carrier. It can be used reference fuel expenses.

# **Question** 2 : Find the percentage of canceled flights for each of the top 5 domestic carriers

In [4]:
## Prepare Data
# mutate column NorC separate data Not cancelled flight and Cancelled flight
f1 <- flights %>%
  mutate(NorC = factor(if_else(!is.na(arr_delay) & !is.na(air_time),"notcancelled","cancelled"),
                       levels = c("cancelled","notcancelled"),
                       labels = c("cancelled","notcancelled"),))
# Find amount of cancelled flights from NorC column
p1 <- f1 %>%
  filter(NorC == "cancelled") %>%
  group_by(carrier,NorC) %>%
  count(NorC, name = "count_cancelled") %>%  
  select(carrier,NorC,count_cancelled) %>%
  arrange(NorC,desc(count_cancelled)) %>%
  ungroup()
# Find Top 5 Domestic Carriers (by flights count)
p2 <- f1 %>%
  filter(NorC == "notcancelled") %>%
  group_by(carrier,NorC) %>%
  count(NorC, name = "count_notcancelled") %>%
  select(carrier,NorC,count_notcancelled) %>%
  arrange(NorC,desc(count_notcancelled)) %>%
  head(5) %>%
  ungroup()

# Find the percentage of canceled flights for each of the top 5 domestic carriers.
j1 <- left_join(p2,p1, by = "carrier")
left_join(j1,airlines, by = "carrier") %>%
  mutate(percent_cancelled = round((count_cancelled/count_notcancelled)*100,digit = 2)) %>%
  mutate(percent_cancelled = paste0(percent_cancelled," %")) %>%
  select(carrier,name,count_notcancelled,count_cancelled,percent_cancelled)

carrier,name,count_notcancelled,count_cancelled,percent_cancelled
<chr>,<chr>,<int>,<int>,<chr>
UA,United Air Lines Inc.,57782,883,1.53 %
B6,JetBlue Airways,54049,586,1.08 %
EV,ExpressJet Airlines Inc.,51108,3065,6 %
DL,Delta Air Lines Inc.,47658,452,0.95 %
AA,American Airlines Inc.,31947,782,2.45 %


**Description :** United Airlines in 2013 amount of flights is 57,782 flight and percentage of cancelled is 1.53 %.

# **Question** 3 :  Find Top 5 aircraft manufacturers

In [5]:
##Prepare Data
# Find Not cancelled flights and clear missing data of tail number column
e1 <- flights %>% 
        filter(!is.na(arr_delay),!is.na(air_time),!is.na(tailnum)) %>%
        select(carrier,flight,tailnum,origin,dest)

# Find top 5 aircraft manufacturers
e2 <- left_join(e1,planes, by = "tailnum")
e2 %>% 
  filter(!is.na(manufacturer)) %>%
  group_by(manufacturer) %>% 
  count(manufacturer, sort = TRUE) %>%
  head(5)

manufacturer,n
<chr>,<int>
BOEING,82283
EMBRAER,63530
AIRBUS,46890
AIRBUS INDUSTRIE,40642
BOMBARDIER INC,27430


# **Question** 4 :  Find average flights per day of each 3 airport

In [6]:
##Prepare Data
# Not cancelled flights
df_nc <- filter(flights,!is.na(arr_time),!is.na(air_time))

# mutate column date from column time_hour
df_nc <- df_nc %>%
  mutate(date = date(time_hour))

# count flights per day
df1 <- df_nc %>%
  group_by(date, origin) %>%
  count(origin) %>%
  arrange(origin)

# Find average flights per day of each 3 airport
df2 <- df1 %>%
  group_by(origin) %>%
  summarise(avg_per_day = round(mean(n),digits = 0)) %>%
  ungroup()

left_join(df2,airports, by = c("origin" = "faa")) %>%
  select(origin,name,avg_per_day)

origin,name,avg_per_day
<chr>,<chr>,<dbl>
EWR,Newark Liberty Intl,320
JFK,John F Kennedy Intl,298
LGA,La Guardia,276


# **Question** 5 :  Find percentage of departure delays of EWR - Newark Liberty International Airport

In [7]:
##Prepare Data
# Not cancelled flights
df_nc <- filter(flights,!is.na(arr_time),!is.na(air_time))

# split departure delays by hours
dt2 <- df_nc %>%
  mutate( hrs_delay = 
            case_when(
              dep_delay < 120 ~ "lower 2 hrs", ## <2
              dep_delay < 180 ~ "2-3 hrs", ## <3
              dep_delay < 300 ~ "3-5 hrs", ## <5
              dep_delay < 360 ~ "5-6 hrs", ## <6
              dep_delay >= 360 ~ "more than 6 hrs"
            ))

# Find percentage of departure delays of EWR - Newark Liberty International Airport
dt2 %>%
  group_by(hrs_delay,origin) %>%
  count(origin, name = "dep_delay2") %>%
  group_by(origin) %>%
  mutate(percentage = dep_delay2/sum(dep_delay2)*100) %>%
  mutate(percentage = round(percentage, digits = 2)) %>%
  mutate(percentage = paste0(percentage," %")) %>%
  filter(origin == "EWR") %>%
  select(origin,hrs_delay,percentage)

origin,hrs_delay,percentage
<chr>,<chr>,<chr>
EWR,2-3 hrs,2.03 %
EWR,3-5 hrs,1.12 %
EWR,5-6 hrs,0.11 %
EWR,lower 2 hrs,96.67 %
EWR,more than 6 hrs,0.07 %


**Description :** In Thailand, If airline is delayed we are able to claim compensation. The delay level will be divided into hours, starting from 2 hours or more.